# 🚀 Entrenamiento de Modelos Transformer para Clasificación de Ocupaciones ENAHO

---

## 📋 Descripción
Script robusto y universal para entrenar modelos de clasificación de texto.

### 🎯 Modelos Soportados:
- **BETO**: `dccuchile/bert-base-spanish-wwm-cased` (Español)
- **XLM-RoBERTa**: `FacebookAI/xlm-roberta-base` (Multilingüe)

### ✨ Características:
- ✅ Cambio de modelo con una sola variable
- ✅ Métricas detalladas (macro, micro, weighted)
- ✅ Manejo robusto de errores
- ✅ Logging detallado para debugging
- ✅ Validación de datos en cada paso
- ✅ Guardado completo de modelo y artefactos
- ✅ Funciones de predicción e inferencia

---

**Autor**: Sistema de Clasificación ENAHO  
**Fecha**: 2025  
**Entorno**: VSCode con Jupyter Notebook  


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# ============================================================================
# INSTALACIÓN DE DEPENDENCIAS (Ejecutar solo una vez)
# ============================================================================

# Descomenta si necesitas instalar las librerías
# !pip install transformers==4.36.0 datasets==2.15.0 scikit-learn==1.3.2
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
# !pip install accelerate sentencepiece

print("✅ Si las librerías ya están instaladas, puedes continuar")


✅ Si las librerías ya están instaladas, puedes continuar


In [3]:
# ============================================================================
# IMPORTACIONES Y VERIFICACIÓN DEL ENTORNO
# ============================================================================

import sys
import os
import warnings
import logging
from datetime import datetime
from pathlib import Path

# Data & ML
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score,
    precision_recall_fscore_support,
    classification_report,
    confusion_matrix
)

# Transformers
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback
)
from torch.utils.data import Dataset

# Utilities
from tqdm.auto import tqdm
import pickle
import json

warnings.filterwarnings('ignore')

# ============================================================================
# CONFIGURACIÓN DE LOGGING
# ============================================================================

def setup_logging(output_dir):
    """Configura el sistema de logging para debugging"""
    os.makedirs(output_dir, exist_ok=True)
    log_file = os.path.join(output_dir, f'training_{datetime.now().strftime("%Y%m%d_%H%M%S")}.log')

    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s',
        handlers=[
            logging.FileHandler(log_file, encoding='utf-8'),
            logging.StreamHandler(sys.stdout)
        ]
    )
    return logging.getLogger(__name__)

# ============================================================================
# VERIFICACIÓN DE GPU
# ============================================================================

print("\n" + "="*80)
print("🔍 VERIFICACIÓN DEL ENTORNO")
print("="*80)

print(f"\n📦 Versiones:")
print(f"   Python: {sys.version.split()[0]}")
print(f"   PyTorch: {torch.__version__}")
print(f"   CUDA disponible: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"\n🎮 GPU Detectada:")
    print(f"   Dispositivo: {torch.cuda.get_device_name(0)}")
    print(f"   Memoria total: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    print(f"   Memoria libre: {(torch.cuda.get_device_properties(0).total_memory - torch.cuda.memory_allocated(0)) / 1e9:.2f} GB")
else:
    print("\n⚠️  GPU no detectada - El entrenamiento será lento")
    print("   Considera usar Google Colab o configurar CUDA")

print("\n" + "="*80)
print("✅ Importaciones completadas correctamente")
print("="*80 + "\n")



🔍 VERIFICACIÓN DEL ENTORNO

📦 Versiones:
   Python: 3.12.12
   PyTorch: 2.8.0+cu126
   CUDA disponible: True

🎮 GPU Detectada:
   Dispositivo: Tesla T4
   Memoria total: 15.83 GB
   Memoria libre: 15.83 GB

✅ Importaciones completadas correctamente



In [4]:
# ============================================================================
# ⚙️  CONFIGURACIÓN PRINCIPAL - MODIFICA AQUÍ
# ============================================================================

class ModelConfig:
    """
    Configuración centralizada del modelo y entrenamiento

    IMPORTANTE: Solo necesitas cambiar MODEL_NAME para entrenar un modelo diferente
    """

    # ========================================================================
    # 🎯 SELECCIÓN DEL MODELO - CAMBIA SOLO ESTA LÍNEA
    # ========================================================================

    # MODEL_NAME = "FacebookAI/xlm-roberta-base"  # Opción 1: XLM-RoBERTa (multilingüe)
    # MODEL_NAME = "dccuchile/bert-base-spanish-wwm-cased"  # Opción 2: BETO (español)
    MODEL_NAME = "bertin-project/bertin-roberta-base-spanish"

    # ========================================================================
    # 📂 RUTAS DE DATOS
    # ========================================================================

    # Ruta al archivo de datos (ajusta según tu ubicación)
    DATA_PATH = "/content/drive/MyDrive/PI_PEU/BASE_LIMPIA_VF.parquet"  # Cambia esta ruta

    # Directorio base para outputs
    BASE_OUTPUT_DIR = "/content/drive/MyDrive/Colab Notebooks/PI_PEU/Robertabne"

    # ========================================================================
    # 📊 COLUMNAS DEL DATASET
    # ========================================================================

    TEXT_COLUMN = "texto_final"  # Columna con el texto
    TARGET_COLUMN = "p505r4"     # Columna objetivo (clase)

    # ========================================================================
    # 🎛️  HIPERPARÁMETROS DE ENTRENAMIENTO
    # ========================================================================

    # Tokenización
    MAX_LENGTH = 128  # Longitud máxima de tokens

    # Entrenamiento
    BATCH_SIZE = 16          # Ajusta según tu GPU (16, 32, 64)
    LEARNING_RATE = 2e-5     # Tasa de aprendizaje
    NUM_EPOCHS = 3           # Número de épocas
    WARMUP_STEPS = 500       # Pasos de warmup
    WEIGHT_DECAY = 0.01      # Regularización

    # División de datos
    TEST_SIZE = 0.15         # 15% para test
    VAL_SIZE = 0.15          # 15% para validación
    RANDOM_STATE = 2025      # Semilla para reproducibilidad

    # Filtrado de clases raras
    MIN_SAMPLES_PER_CLASS = 10  # Mínimo de muestras por clase

    # Early stopping
    EARLY_STOPPING_PATIENCE = 3

    # ========================================================================
    # 🔧 CONFIGURACIÓN AUTOMÁTICA (NO MODIFICAR)
    # ========================================================================

    def __init__(self):
        """Inicializa configuración y crea directorios"""
        # Detectar tipo de modelo del nombre
        if "roberta" in self.MODEL_NAME.lower():
            self.model_type = "xlm-roberta"
        elif "bert" in self.MODEL_NAME.lower():
            self.model_type = "bert"
        else:
            self.model_type = "transformer"

        # Crear nombre descriptivo para el experimento
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        model_short_name = self.MODEL_NAME.split('/')[-1]
        self.experiment_name = f"{model_short_name}_{timestamp}"

        # Configurar directorios
        self.OUTPUT_DIR = os.path.join(self.BASE_OUTPUT_DIR, self.experiment_name)
        self.MODEL_SAVE_DIR = os.path.join(self.OUTPUT_DIR, "final_model")
        self.CHECKPOINT_DIR = os.path.join(self.OUTPUT_DIR, "checkpoints")

        # Crear directorios
        for dir_path in [self.OUTPUT_DIR, self.MODEL_SAVE_DIR, self.CHECKPOINT_DIR]:
            os.makedirs(dir_path, exist_ok=True)

        # Device
        self.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

        # Configurar logging
        self.logger = setup_logging(self.OUTPUT_DIR)
        self.logger.info(f"Experimento iniciado: {self.experiment_name}")
        self.logger.info(f"Modelo seleccionado: {self.MODEL_NAME}")
        self.logger.info(f"Dispositivo: {self.DEVICE}")

    def display_config(self):
        """Muestra la configuración actual"""
        print("\n" + "="*80)
        print("⚙️  CONFIGURACIÓN DEL MODELO")
        print("="*80)
        print(f"\n🤖 Modelo: {self.MODEL_NAME}")
        print(f"   Tipo: {self.model_type}")
        print(f"   Experimento: {self.experiment_name}")
        print(f"\n📂 Rutas:")
        print(f"   Datos: {self.DATA_PATH}")
        print(f"   Output: {self.OUTPUT_DIR}")
        print(f"   Modelo final: {self.MODEL_SAVE_DIR}")
        print(f"\n📊 Datos:")
        print(f"   Columna texto: {self.TEXT_COLUMN}")
        print(f"   Columna target: {self.TARGET_COLUMN}")
        print(f"   Max length: {self.MAX_LENGTH}")
        print(f"\n🎛️  Entrenamiento:")
        print(f"   Batch size: {self.BATCH_SIZE}")
        print(f"   Learning rate: {self.LEARNING_RATE}")
        print(f"   Epochs: {self.NUM_EPOCHS}")
        print(f"   Early stopping: {self.EARLY_STOPPING_PATIENCE} epochs")
        print(f"\n💾 División de datos:")
        print(f"   Test: {self.TEST_SIZE*100:.0f}%")
        print(f"   Validación: {self.VAL_SIZE*100:.0f}%")
        print(f"   Train: {(1-self.TEST_SIZE-self.VAL_SIZE)*100:.0f}%")
        print("\n" + "="*80 + "\n")

    def save_config(self):
        """Guarda la configuración en JSON"""
        config_dict = {
            'model_name': self.MODEL_NAME,
            'model_type': self.model_type,
            'experiment_name': self.experiment_name,
            'data_path': self.DATA_PATH,
            'text_column': self.TEXT_COLUMN,
            'target_column': self.TARGET_COLUMN,
            'max_length': self.MAX_LENGTH,
            'batch_size': self.BATCH_SIZE,
            'learning_rate': self.LEARNING_RATE,
            'num_epochs': self.NUM_EPOCHS,
            'test_size': self.TEST_SIZE,
            'val_size': self.VAL_SIZE,
            'random_state': self.RANDOM_STATE,
            'min_samples_per_class': self.MIN_SAMPLES_PER_CLASS,
            'device': self.DEVICE,
            'timestamp': datetime.now().isoformat()
        }

        config_path = os.path.join(self.OUTPUT_DIR, 'config.json')
        with open(config_path, 'w', encoding='utf-8') as f:
            json.dump(config_dict, f, indent=2, ensure_ascii=False)

        self.logger.info(f"Configuración guardada en: {config_path}")
        return config_path


# Inicializar configuración
config = ModelConfig()
config.display_config()
config.save_config()



⚙️  CONFIGURACIÓN DEL MODELO

🤖 Modelo: bertin-project/bertin-roberta-base-spanish
   Tipo: xlm-roberta
   Experimento: bertin-roberta-base-spanish_20251112_225626

📂 Rutas:
   Datos: /content/drive/MyDrive/PI_PEU/BASE_LIMPIA_VF.parquet
   Output: /content/drive/MyDrive/Colab Notebooks/PI_PEU/Robertabne/bertin-roberta-base-spanish_20251112_225626
   Modelo final: /content/drive/MyDrive/Colab Notebooks/PI_PEU/Robertabne/bertin-roberta-base-spanish_20251112_225626/final_model

📊 Datos:
   Columna texto: texto_final
   Columna target: p505r4
   Max length: 128

🎛️  Entrenamiento:
   Batch size: 16
   Learning rate: 2e-05
   Epochs: 3
   Early stopping: 3 epochs

💾 División de datos:
   Test: 15%
   Validación: 15%
   Train: 70%




'/content/drive/MyDrive/Colab Notebooks/PI_PEU/Robertabne/bertin-roberta-base-spanish_20251112_225626/config.json'

In [5]:
# ============================================================================
# 📂 CARGA Y VALIDACIÓN DE DATOS
# ============================================================================

class DataLoader:
    """Cargador y validador de datos con manejo robusto de errores"""

    def __init__(self, config):
        self.config = config
        self.logger = config.logger

    def load_data(self):
        """
        Carga datos desde archivo con validación

        Returns:
            pd.DataFrame: Datos cargados
        """
        try:
            self.logger.info(f"Cargando datos desde: {self.config.DATA_PATH}")

            # Verificar que el archivo existe
            if not os.path.exists(self.config.DATA_PATH):
                raise FileNotFoundError(
                    f"❌ El archivo no existe: {self.config.DATA_PATH}\n"
                    f"   Por favor, verifica la ruta en ModelConfig.DATA_PATH"
                )

            # Cargar según extensión
            file_ext = os.path.splitext(self.config.DATA_PATH)[1].lower()

            if file_ext == '.parquet':
                df = pd.read_parquet(self.config.DATA_PATH)
            elif file_ext == '.csv':
                df = pd.read_csv(self.config.DATA_PATH)
            elif file_ext in ['.xlsx', '.xls']:
                df = pd.read_excel(self.config.DATA_PATH)
            else:
                raise ValueError(
                    f"❌ Formato no soportado: {file_ext}\n"
                    f"   Formatos válidos: .parquet, .csv, .xlsx, .xls"
                )

            self.logger.info(f"✅ Datos cargados: {df.shape[0]:,} filas x {df.shape[1]} columnas")

            return df

        except Exception as e:
            self.logger.error(f"❌ Error al cargar datos: {str(e)}")
            raise

    def validate_data(self, df):
        """
        Valida que los datos tengan las columnas necesarias

        Args:
            df: DataFrame a validar

        Raises:
            ValueError: Si faltan columnas requeridas
        """
        self.logger.info("Validando estructura de datos...")

        # Verificar columnas requeridas
        required_cols = [self.config.TEXT_COLUMN, self.config.TARGET_COLUMN]
        missing_cols = [col for col in required_cols if col not in df.columns]

        if missing_cols:
            available_cols = list(df.columns)
            raise ValueError(
                f"❌ Columnas faltantes: {missing_cols}\n"
                f"   Columnas disponibles: {available_cols}\n"
                f"   Verifica TEXT_COLUMN y TARGET_COLUMN en ModelConfig"
            )

        # Validar datos no nulos
        null_text = df[self.config.TEXT_COLUMN].isna().sum()
        null_target = df[self.config.TARGET_COLUMN].isna().sum()

        self.logger.info(
            f"   Valores nulos - Texto: {null_text:,}, Target: {null_target:,}"
        )

        # Validar textos vacíos
        empty_text = (df[self.config.TEXT_COLUMN].str.strip() == '').sum()
        if empty_text > 0:
            self.logger.warning(f"   ⚠️  Textos vacíos: {empty_text:,}")

        self.logger.info("✅ Validación completada")

    def filter_valid_records(self, df):
        """
        Filtra registros válidos (no nulos, no vacíos)

        Args:
            df: DataFrame original

        Returns:
            pd.DataFrame: DataFrame filtrado
        """
        self.logger.info("Filtrando registros válidos...")

        initial_count = len(df)

        # Filtrar nulos y vacíos
        df_clean = df[
            df[self.config.TEXT_COLUMN].notna() &
            df[self.config.TARGET_COLUMN].notna() &
            (df[self.config.TEXT_COLUMN].str.strip() != '')
        ].copy()

        final_count = len(df_clean)
        removed = initial_count - final_count

        self.logger.info(
            f"   Registros iniciales: {initial_count:,}\n"
            f"   Registros válidos: {final_count:,}\n"
            f"   Removidos: {removed:,} ({removed/initial_count*100:.2f}%)"
        )

        if final_count == 0:
            raise ValueError(
                "❌ No quedan registros válidos después del filtrado\n"
                "   Verifica la calidad de tus datos"
            )

        return df_clean

    def filter_rare_classes(self, df):
        """
        Filtra clases con pocas muestras

        Args:
            df: DataFrame

        Returns:
            pd.DataFrame: DataFrame filtrado
        """
        self.logger.info(
            f"Filtrando clases con < {self.config.MIN_SAMPLES_PER_CLASS} muestras..."
        )

        # Contar muestras por clase
        class_counts = df[self.config.TARGET_COLUMN].value_counts()

        # Identificar clases válidas
        valid_classes = class_counts[class_counts >= self.config.MIN_SAMPLES_PER_CLASS].index
        rare_classes = class_counts[class_counts < self.config.MIN_SAMPLES_PER_CLASS]

        # Filtrar
        df_filtered = df[df[self.config.TARGET_COLUMN].isin(valid_classes)].copy()

        self.logger.info(
            f"   Clases originales: {len(class_counts):,}\n"
            f"   Clases mantenidas: {len(valid_classes):,}\n"
            f"   Clases removidas: {len(rare_classes):,}\n"
            f"   Registros antes: {len(df):,}\n"
            f"   Registros después: {len(df_filtered):,}"
        )

        if len(df_filtered) == 0:
            raise ValueError(
                f"❌ No quedan registros después de filtrar clases raras\n"
                f"   Considera reducir MIN_SAMPLES_PER_CLASS"
            )

        return df_filtered

    def create_label_mapping(self, df):
        """
        Crea mapeo de etiquetas a índices

        Args:
            df: DataFrame

        Returns:
            tuple: (df_with_labels, label2id, id2label)
        """
        self.logger.info("Creando mapeo de etiquetas...")

        # Obtener clases únicas ordenadas
        unique_labels = sorted(df[self.config.TARGET_COLUMN].unique())

        # Crear mapeos
        label2id = {label: idx for idx, label in enumerate(unique_labels)}
        id2label = {idx: label for label, idx in label2id.items()}

        # Agregar columna de índices numéricos
        df['label_id'] = df[self.config.TARGET_COLUMN].map(label2id)

        # Verificar que no hay nulos (no debería pasar)
        if df['label_id'].isna().any():
            raise ValueError("❌ Error en el mapeo de etiquetas")

        self.logger.info(
            f"✅ Mapeo creado: {len(label2id)} clases (índices 0-{len(label2id)-1})"
        )

        # Mostrar distribución de clases
        class_dist = df[self.config.TARGET_COLUMN].value_counts()
        self.logger.info(
            f"   Clase más frecuente: {class_dist.index[0]} ({class_dist.iloc[0]:,} muestras)\n"
            f"   Clase menos frecuente: {class_dist.index[-1]} ({class_dist.iloc[-1]:,} muestras)\n"
            f"   Promedio por clase: {class_dist.mean():.1f}"
        )

        return df, label2id, id2label

    def split_data(self, df):
        """
        Divide datos en train, validation y test con estratificación

        Args:
            df: DataFrame

        Returns:
            tuple: (train_df, val_df, test_df)
        """
        self.logger.info("Dividiendo datos...")

        try:
            # Primero separar test
            train_val, test = train_test_split(
                df,
                test_size=self.config.TEST_SIZE,
                random_state=self.config.RANDOM_STATE,
                stratify=df['label_id']
            )

            # Luego separar train y validation
            val_size_adjusted = self.config.VAL_SIZE / (1 - self.config.TEST_SIZE)
            train, val = train_test_split(
                train_val,
                test_size=val_size_adjusted,
                random_state=self.config.RANDOM_STATE,
                stratify=train_val['label_id']
            )

            self.logger.info(
                f"✅ División completada:\n"
                f"   Train: {len(train):,} ({len(train)/len(df)*100:.1f}%)\n"
                f"   Validation: {len(val):,} ({len(val)/len(df)*100:.1f}%)\n"
                f"   Test: {len(test):,} ({len(test)/len(df)*100:.1f}%)"
            )

            return train, val, test

        except ValueError as e:
            self.logger.error(
                f"❌ Error al dividir datos: {str(e)}\n"
                f"   Puede ser que algunas clases tengan muy pocas muestras\n"
                f"   Considera aumentar MIN_SAMPLES_PER_CLASS"
            )
            raise


# ============================================================================
# EJECUTAR CARGA DE DATOS
# ============================================================================

print("\n" + "="*80)
print("📂 CARGANDO Y PREPARANDO DATOS")
print("="*80 + "\n")

try:
    # Inicializar cargador
    data_loader = DataLoader(config)

    # Cargar datos
    df_raw = data_loader.load_data()

    # Validar estructura
    data_loader.validate_data(df_raw)

    # Filtrar registros válidos
    df_valid = data_loader.filter_valid_records(df_raw)

    # Filtrar clases raras
    df_filtered = data_loader.filter_rare_classes(df_valid)

    # Crear mapeo de etiquetas
    df_final, label2id, id2label = data_loader.create_label_mapping(df_filtered)

    # Dividir datos
    train_df, val_df, test_df = data_loader.split_data(df_final)

    print("\n" + "="*80)
    print("✅ DATOS PREPARADOS EXITOSAMENTE")
    print("="*80 + "\n")

    # Guardar información de las clases
    class_info = {
        'num_classes': len(label2id),
        'label2id': label2id,
        'id2label': id2label,
        'class_distribution': df_final[config.TARGET_COLUMN].value_counts().to_dict()
    }

    with open(os.path.join(config.OUTPUT_DIR, 'class_info.json'), 'w', encoding='utf-8') as f:
        json.dump(class_info, f, indent=2, ensure_ascii=False)

except Exception as e:
    print("\n" + "="*80)
    print("❌ ERROR EN LA CARGA DE DATOS")
    print("="*80)
    print(f"\n{str(e)}\n")
    print("Por favor, revisa:")
    print("1. La ruta del archivo en ModelConfig.DATA_PATH")
    print("2. Los nombres de columnas en TEXT_COLUMN y TARGET_COLUMN")
    print("3. La calidad de tus datos (nulos, vacíos, etc.)")
    print("\n" + "="*80 + "\n")
    raise



📂 CARGANDO Y PREPARANDO DATOS


✅ DATOS PREPARADOS EXITOSAMENTE



In [6]:
# ============================================================================
# 🔤 DATASET Y TOKENIZACIÓN
# ============================================================================

class TextClassificationDataset(Dataset):
    """
    Dataset personalizado para clasificación de texto
    Compatible con cualquier modelo de Hugging Face
    """

    def __init__(self, texts, labels, tokenizer, max_length):
        """
        Args:
            texts: Lista de textos
            labels: Lista de etiquetas (índices numéricos)
            tokenizer: Tokenizer de Hugging Face
            max_length: Longitud máxima de tokens
        """
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        """
        Obtiene un ejemplo tokenizado

        Returns:
            dict: Diccionario con input_ids, attention_mask y labels
        """
        text = str(self.texts[idx])
        label = int(self.labels[idx])

        # Tokenizar
        encoding = self.tokenizer(
            text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }


# ============================================================================
# INICIALIZAR TOKENIZER Y DATASETS
# ============================================================================

print("\n" + "="*80)
print("🔤 INICIALIZANDO TOKENIZER Y DATASETS")
print("="*80 + "\n")

try:
    # Cargar tokenizer
    config.logger.info(f"Cargando tokenizer: {config.MODEL_NAME}")
    tokenizer = AutoTokenizer.from_pretrained(config.MODEL_NAME)

    print(f"✅ Tokenizer cargado: {config.MODEL_NAME}")
    print(f"   Vocabulario: {len(tokenizer):,} tokens")
    print(f"   Tipo: {tokenizer.__class__.__name__}")

    # Crear datasets
    config.logger.info("Creando datasets...")

    train_dataset = TextClassificationDataset(
        texts=train_df[config.TEXT_COLUMN].tolist(),
        labels=train_df['label_id'].tolist(),
        tokenizer=tokenizer,
        max_length=config.MAX_LENGTH
    )

    val_dataset = TextClassificationDataset(
        texts=val_df[config.TEXT_COLUMN].tolist(),
        labels=val_df['label_id'].tolist(),
        tokenizer=tokenizer,
        max_length=config.MAX_LENGTH
    )

    test_dataset = TextClassificationDataset(
        texts=test_df[config.TEXT_COLUMN].tolist(),
        labels=test_df['label_id'].tolist(),
        tokenizer=tokenizer,
        max_length=config.MAX_LENGTH
    )

    print(f"\n✅ Datasets creados:")
    print(f"   Train: {len(train_dataset):,} ejemplos")
    print(f"   Validation: {len(val_dataset):,} ejemplos")
    print(f"   Test: {len(test_dataset):,} ejemplos")

    # Verificar un ejemplo
    sample = train_dataset[0]
    print(f"\n📝 Ejemplo de muestra tokenizada:")
    print(f"   Input IDs shape: {sample['input_ids'].shape}")
    print(f"   Attention mask shape: {sample['attention_mask'].shape}")
    print(f"   Label: {sample['labels'].item()}")

    print("\n" + "="*80)
    print("✅ TOKENIZACIÓN COMPLETADA")
    print("="*80 + "\n")

except Exception as e:
    print("\n" + "="*80)
    print("❌ ERROR EN LA TOKENIZACIÓN")
    print("="*80)
    print(f"\n{str(e)}\n")
    print("Posibles causas:")
    print("1. El modelo no está disponible (verifica MODEL_NAME)")
    print("2. No hay conexión a internet para descargar el tokenizer")
    print("3. Problema con los datos de entrada")
    print("\n" + "="*80 + "\n")
    raise



🔤 INICIALIZANDO TOKENIZER Y DATASETS



tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

✅ Tokenizer cargado: bertin-project/bertin-roberta-base-spanish
   Vocabulario: 50,262 tokens
   Tipo: RobertaTokenizerFast

✅ Datasets creados:
   Train: 220,937 ejemplos
   Validation: 47,344 ejemplos
   Test: 47,344 ejemplos

📝 Ejemplo de muestra tokenizada:
   Input IDs shape: torch.Size([128])
   Attention mask shape: torch.Size([128])
   Label: 356

✅ TOKENIZACIÓN COMPLETADA



In [7]:
# ============================================================================
# 📊 MÉTRICAS DETALLADAS
# ============================================================================

def compute_detailed_metrics(eval_pred):
    """
    Calcula métricas detalladas: Accuracy, Precision, Recall, F1
    Con variantes: macro, micro y weighted

    Args:
        eval_pred: Predicciones del modelo (predictions, label_ids)

    Returns:
        dict: Diccionario con todas las métricas
    """
    predictions, labels = eval_pred

    # Obtener predicciones (argmax si son logits)
    if predictions.ndim > 1:
        preds = np.argmax(predictions, axis=1)
    else:
        preds = predictions

    # Accuracy
    accuracy = accuracy_score(labels, preds)

    # Precision, Recall, F1 con diferentes promedios
    # Macro: promedio sin ponderar (todas las clases tienen el mismo peso)
    precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(
        labels, preds, average='macro', zero_division=0
    )

    # Micro: agregado global (considera todas las muestras por igual)
    precision_micro, recall_micro, f1_micro, _ = precision_recall_fscore_support(
        labels, preds, average='micro', zero_division=0
    )

    # Weighted: promedio ponderado por soporte de cada clase
    precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(
        labels, preds, average='weighted', zero_division=0
    )

    return {
        # Accuracy (solo una versión)
        'accuracy': accuracy,

        # F1 Score
        'f1_macro': f1_macro,
        'f1_micro': f1_micro,
        'f1_weighted': f1_weighted,

        # Precision
        'precision_macro': precision_macro,
        'precision_micro': precision_micro,
        'precision_weighted': precision_weighted,

        # Recall
        'recall_macro': recall_macro,
        'recall_micro': recall_micro,
        'recall_weighted': recall_weighted,
    }



def display_metrics(metrics, title="Métricas"):
    """Muestra las métricas de forma organizada"""
    print("\n" + "="*80)
    print(f"📊 {title.upper()}")
    print("="*80 + "\n")

    # Mostrar Loss si existe
    loss = (
        metrics.get('test_loss') or
        metrics.get('eval_loss') or
        metrics.get('loss', None)
    )
    if loss is not None:
        print(f"💥 LOSS: {loss:.4f}")

    # Accuracy
    acc = (
        metrics.get('test_accuracy') or
        metrics.get('eval_accuracy') or
        metrics.get('accuracy', 0)
    )
    print(f"🎯 ACCURACY: {acc:.4f}")
    print("\n" + "-"*80)

    # Tabla
    print(f"\n{'Métrica':<20} {'Macro':>12} {'Micro':>12} {'Weighted':>12}")
    print("-"*60)

    def get_m(name):
        return (
            metrics.get(f'test_{name}') or
            metrics.get(f'eval_{name}') or
            metrics.get(name, 0)
        )

    print(f"{'F1 Score':<20} {get_m('f1_macro'):>12.4f} {get_m('f1_micro'):>12.4f} {get_m('f1_weighted'):>12.4f}")
    print(f"{'Precision':<20} {get_m('precision_macro'):>12.4f} {get_m('precision_micro'):>12.4f} {get_m('precision_weighted'):>12.4f}")
    print(f"{'Recall':<20} {get_m('recall_macro'):>12.4f} {get_m('recall_micro'):>12.4f} {get_m('recall_weighted'):>12.4f}")

    print("\n" + "="*80 + "\n")


print("✅ Funciones de métricas cargadas (con loss incluido)")


✅ Funciones de métricas cargadas (con loss incluido)


In [8]:
# ============================================================================
# 🤖 INICIALIZACIÓN DEL MODELO
# ============================================================================

print("\n" + "="*80)
print("🤖 INICIALIZANDO MODELO")
print("="*80 + "\n")

try:
    config.logger.info(f"Cargando modelo: {config.MODEL_NAME}")

    # Cargar modelo
    model = AutoModelForSequenceClassification.from_pretrained(
        config.MODEL_NAME,
        num_labels=len(label2id),
        id2label=id2label,
        label2id=label2id,
        problem_type="single_label_classification"
    )

    # Mover a GPU si está disponible
    model.to(config.DEVICE)

    # Información del modelo
    num_params = sum(p.numel() for p in model.parameters())
    num_trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)

    print(f"✅ Modelo cargado: {config.MODEL_NAME}")
    print(f"   Tipo: {model.__class__.__name__}")
    print(f"   Número de clases: {len(label2id)}")
    print(f"   Parámetros totales: {num_params:,}")
    print(f"   Parámetros entrenables: {num_trainable:,}")
    print(f"   Dispositivo: {config.DEVICE}")

    if torch.cuda.is_available():
        print(f"   Memoria GPU asignada: {torch.cuda.memory_allocated(0) / 1e9:.2f} GB")

    config.logger.info(f"Modelo inicializado con {num_params:,} parámetros")

    print("\n" + "="*80)
    print("✅ MODELO LISTO PARA ENTRENAMIENTO")
    print("="*80 + "\n")

except Exception as e:
    print("\n" + "="*80)
    print("❌ ERROR AL CARGAR EL MODELO")
    print("="*80)
    print(f"\n{str(e)}\n")
    print("Posibles causas:")
    print("1. El nombre del modelo es incorrecto")
    print("2. No hay conexión a internet para descargar el modelo")
    print("3. No hay suficiente memoria GPU/RAM")
    print("4. Incompatibilidad de versiones de transformers")
    print("\nModelos válidos:")
    print("- FacebookAI/xlm-roberta-base")
    print("- dccuchile/bert-base-spanish-wwm-cased")
    print("\n" + "="*80 + "\n")
    raise



🤖 INICIALIZANDO MODELO



config.json:   0%|          | 0.00/674 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at bertin-project/bertin-roberta-base-spanish and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Modelo cargado: bertin-project/bertin-roberta-base-spanish
   Tipo: RobertaForSequenceClassification
   Número de clases: 357
   Parámetros totales: 124,917,861
   Parámetros entrenables: 124,917,861
   Dispositivo: cuda
   Memoria GPU asignada: 0.50 GB

✅ MODELO LISTO PARA ENTRENAMIENTO



In [9]:
# ============================================================================
# ⚙️  CONFIGURACIÓN DEL ENTRENAMIENTO
# ============================================================================

print("\n" + "="*80)
print("⚙️  CONFIGURANDO ENTRENAMIENTO")
print("="*80 + "\n")

# Configuración de argumentos de entrenamiento
training_args = TrainingArguments(
    # Directorios
    output_dir=config.CHECKPOINT_DIR,
    logging_dir=os.path.join(config.OUTPUT_DIR, 'logs'),

    # Hiperparámetros
    learning_rate=config.LEARNING_RATE,
    per_device_train_batch_size=config.BATCH_SIZE,
    per_device_eval_batch_size=config.BATCH_SIZE,
    num_train_epochs=config.NUM_EPOCHS,
    warmup_steps=config.WARMUP_STEPS,
    weight_decay=config.WEIGHT_DECAY,

    # Evaluación y guardado
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1_weighted",  # Usar F1 weighted como métrica principal
    greater_is_better=True,

    # Logging
    logging_steps=100,
    logging_strategy="steps",

    # Optimización
    fp16=torch.cuda.is_available(),  # Precisión mixta si hay GPU
    gradient_accumulation_steps=1,

    # Otros
    seed=config.RANDOM_STATE,
    report_to="none",  # Desactivar reportes externos
    disable_tqdm=False,  # Mantener barra de progreso
)

# Inicializar Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_detailed_metrics,
    callbacks=[
        EarlyStoppingCallback(
            early_stopping_patience=config.EARLY_STOPPING_PATIENCE
        )
    ]
)

print("✅ Configuración de entrenamiento:")
print(f"   Learning rate: {config.LEARNING_RATE}")
print(f"   Batch size: {config.BATCH_SIZE}")
print(f"   Epochs: {config.NUM_EPOCHS}")
print(f"   Warmup steps: {config.WARMUP_STEPS}")
print(f"   Early stopping: {config.EARLY_STOPPING_PATIENCE} epochs")
print(f"   FP16 (mixed precision): {training_args.fp16}")
print(f"   Métrica principal: f1_weighted")

print("\n" + "="*80)
print("✅ TRAINER CONFIGURADO Y LISTO")
print("="*80 + "\n")

config.logger.info("Trainer configurado correctamente")



⚙️  CONFIGURANDO ENTRENAMIENTO

✅ Configuración de entrenamiento:
   Learning rate: 2e-05
   Batch size: 16
   Epochs: 3
   Warmup steps: 500
   Early stopping: 3 epochs
   FP16 (mixed precision): True
   Métrica principal: f1_weighted

✅ TRAINER CONFIGURADO Y LISTO



In [10]:
# ============================================================================
# 🚀 ENTRENAMIENTO DEL MODELO
# ============================================================================

print("\n" + "="*80)
print("🚀 INICIANDO ENTRENAMIENTO")
print("="*80)
print(f"\nModelo: {config.MODEL_NAME}")
print(f"Datos de entrenamiento: {len(train_dataset):,} ejemplos")
print(f"Datos de validación: {len(val_dataset):,} ejemplos")
print(f"\nEsto puede tomar varios minutos/horas dependiendo de:")
print("  - Tamaño del dataset")
print("  - Número de épocas")
print("  - Hardware disponible (GPU/CPU)")
print("\n" + "="*80 + "\n")

try:
    # Registrar inicio
    start_time = datetime.now()
    config.logger.info("Iniciando entrenamiento...")

    # ENTRENAR
    train_result = trainer.train()

    # Registrar finalización
    end_time = datetime.now()
    training_time = end_time - start_time

    config.logger.info(f"Entrenamiento completado en {training_time}")

    print("\n" + "="*80)
    print("✅ ENTRENAMIENTO COMPLETADO")
    print("="*80)
    print(f"\nTiempo total: {training_time}")
    print(f"Mejor modelo guardado en: {config.CHECKPOINT_DIR}")

    # Mostrar métricas finales de entrenamiento
    print(f"\n📊 Métricas finales de entrenamiento:")
    print(f"   Training loss: {train_result.training_loss:.4f}")

    # Evaluar en validation set
    print("\n" + "-"*80)
    print("📊 Evaluando en conjunto de validación...")
    val_metrics = trainer.evaluate()
    display_metrics(val_metrics, "Métricas de Validación")

    print("="*80 + "\n")

except KeyboardInterrupt:
    print("\n" + "="*80)
    print("⚠️  ENTRENAMIENTO INTERRUMPIDO POR EL USUARIO")
    print("="*80)
    print("\nEl modelo puede haber sido parcialmente entrenado.")
    print("Los checkpoints guardados están disponibles en:")
    print(f"  {config.CHECKPOINT_DIR}")
    print("\n" + "="*80 + "\n")
    raise

except Exception as e:
    print("\n" + "="*80)
    print("❌ ERROR DURANTE EL ENTRENAMIENTO")
    print("="*80)
    print(f"\n{str(e)}\n")
    print("Posibles causas:")
    print("1. Memoria insuficiente (GPU/RAM)")
    print("   Solución: Reduce BATCH_SIZE en ModelConfig")
    print("2. Datos corruptos o formato incorrecto")
    print("3. Incompatibilidad de versiones")
    print("\nRevisa el archivo de log para más detalles:")
    print(f"  {os.path.join(config.OUTPUT_DIR, 'training_*.log')}")
    print("\n" + "="*80 + "\n")
    config.logger.error(f"Error en entrenamiento: {str(e)}", exc_info=True)
    raise



🚀 INICIANDO ENTRENAMIENTO

Modelo: bertin-project/bertin-roberta-base-spanish
Datos de entrenamiento: 220,937 ejemplos
Datos de validación: 47,344 ejemplos

Esto puede tomar varios minutos/horas dependiendo de:
  - Tamaño del dataset
  - Número de épocas
  - Hardware disponible (GPU/CPU)




Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,F1 Micro,F1 Weighted,Precision Macro,Precision Micro,Precision Weighted,Recall Macro,Recall Micro,Recall Weighted
1,0.305000,0.330462,0.928798,0.497080,0.928798,0.921912,0.515937,0.928798,0.919186,0.504121,0.928798,0.928798
2,0.258600,0.287590,0.936127,0.584992,0.936127,0.932253,0.603218,0.936127,0.931324,0.590769,0.936127,0.936127


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,F1 Micro,F1 Weighted,Precision Macro,Precision Micro,Precision Weighted,Recall Macro,Recall Micro,Recall Weighted
1,0.305000,0.330462,0.928798,0.497080,0.928798,0.921912,0.515937,0.928798,0.919186,0.504121,0.928798,0.928798
2,0.258600,0.287590,0.936127,0.584992,0.936127,0.932253,0.603218,0.936127,0.931324,0.590769,0.936127,0.936127
3,0.187200,0.286277,0.939654,0.617387,0.939654,0.936650,0.644770,0.939654,0.935782,0.618170,0.939654,0.939654



✅ ENTRENAMIENTO COMPLETADO

Tiempo total: 1:29:33.782914
Mejor modelo guardado en: /content/drive/MyDrive/Colab Notebooks/PI_PEU/Robertabne/bertin-roberta-base-spanish_20251112_225626/checkpoints

📊 Métricas finales de entrenamiento:
   Training loss: 0.3598

--------------------------------------------------------------------------------
📊 Evaluando en conjunto de validación...



📊 MÉTRICAS DE VALIDACIÓN

💥 LOSS: 0.2863
🎯 ACCURACY: 0.9397

--------------------------------------------------------------------------------

Métrica                     Macro        Micro     Weighted
------------------------------------------------------------
F1 Score                   0.6174       0.9397       0.9367
Precision                  0.6448       0.9397       0.9358
Recall                     0.6182       0.9397       0.9397





In [11]:
# ============================================================================
# 🧪 EVALUACIÓN EN TEST SET
# ============================================================================

print("\n" + "="*80)
print("🧪 EVALUACIÓN EN TEST SET")
print("="*80 + "\n")

try:
    config.logger.info("Evaluando en test set...")

    # Obtener predicciones en test set
    test_predictions = trainer.predict(test_dataset)

    # Extraer métricas
    test_metrics = test_predictions.metrics

    # Mostrar métricas
    display_metrics(test_metrics, "Métricas de Test (Evaluación Final)")

    # Guardar métricas en archivo
    metrics_file = os.path.join(config.OUTPUT_DIR, 'test_metrics.json')
    with open(metrics_file, 'w', encoding='utf-8') as f:
        json.dump(test_metrics, f, indent=2)

    config.logger.info(f"Métricas de test guardadas en: {metrics_file}")

    # ========================================================================
    # ANÁLISIS DETALLADO
    # ========================================================================

    print("\n" + "="*80)
    print("📈 ANÁLISIS DETALLADO")
    print("="*80 + "\n")

    # Obtener predicciones y etiquetas verdaderas
    y_pred = np.argmax(test_predictions.predictions, axis=1)
    y_true = test_predictions.label_ids

    # Reporte de clasificación por clase
    print("📊 Reporte de Clasificación por Clase:\n")

    # Crear reporte con nombres de clases
    target_names = [id2label[i] for i in range(len(id2label))]
    class_report = classification_report(
        y_true,
        y_pred,
        target_names=target_names,
        zero_division=0,
        digits=4
    )
    print(class_report)

    # Guardar reporte completo
    report_file = os.path.join(config.OUTPUT_DIR, 'classification_report.txt')
    with open(report_file, 'w', encoding='utf-8') as f:
        f.write("REPORTE DE CLASIFICACIÓN - TEST SET\n")
        f.write("="*80 + "\n\n")
        f.write(f"Modelo: {config.MODEL_NAME}\n")
        f.write(f"Fecha: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"Dataset: {config.DATA_PATH}\n")
        f.write("\n" + "="*80 + "\n\n")
        f.write(class_report)

    print(f"\n✅ Reporte completo guardado en: {report_file}")

    # ========================================================================
    # ANÁLISIS DE ERRORES
    # ========================================================================

    print("\n" + "="*80)
    print("🔍 ANÁLISIS DE ERRORES")
    print("="*80 + "\n")

    # Identificar predicciones incorrectas
    incorrect_mask = y_pred != y_true
    num_incorrect = incorrect_mask.sum()
    error_rate = num_incorrect / len(y_true) * 100

    print(f"Total de predicciones: {len(y_true):,}")
    print(f"Predicciones correctas: {(~incorrect_mask).sum():,}")
    print(f"Predicciones incorrectas: {num_incorrect:,}")
    print(f"Tasa de error: {error_rate:.2f}%")

    # Crear DataFrame con errores
    errors_df = test_df[incorrect_mask].copy()
    errors_df['predicted_label'] = [id2label[pred] for pred in y_pred[incorrect_mask]]
    errors_df['true_label'] = [id2label[true] for true in y_true[incorrect_mask]]
    errors_df['predicted_id'] = y_pred[incorrect_mask]
    errors_df['true_id'] = y_true[incorrect_mask]

    # Agregar probabilidades
    probs = torch.nn.functional.softmax(torch.tensor(test_predictions.predictions), dim=-1)
    max_probs = probs.max(dim=-1).values.numpy()
    errors_df['confidence'] = max_probs[incorrect_mask]

    # Guardar análisis de errores
    errors_file = os.path.join(config.OUTPUT_DIR, 'error_analysis.csv')
    errors_df.to_csv(errors_file, index=False, encoding='utf-8')

    print(f"\n✅ Análisis de errores guardado en: {errors_file}")

    # Mostrar clases con más errores
    if len(errors_df) > 0:
        print("\n📊 Top 10 clases con más errores de predicción:")
        error_by_class = errors_df['true_label'].value_counts().head(10)
        for idx, (clase, count) in enumerate(error_by_class.items(), 1):
            print(f"   {idx}. {clase}: {count} errores")

    print("\n" + "="*80 + "\n")

    config.logger.info("Evaluación en test completada")

except Exception as e:
    print("\n" + "="*80)
    print("❌ ERROR EN LA EVALUACIÓN")
    print("="*80)
    print(f"\n{str(e)}\n")
    config.logger.error(f"Error en evaluación: {str(e)}", exc_info=True)
    raise



🧪 EVALUACIÓN EN TEST SET




📊 MÉTRICAS DE TEST (EVALUACIÓN FINAL)

💥 LOSS: 0.2834
🎯 ACCURACY: 0.9402

--------------------------------------------------------------------------------

Métrica                     Macro        Micro     Weighted
------------------------------------------------------------
F1 Score                   0.6157       0.9402       0.9371
Precision                  0.6416       0.9402       0.9366
Recall                     0.6217       0.9402       0.9402



📈 ANÁLISIS DETALLADO

📊 Reporte de Clasificación por Clase:

              precision    recall  f1-score   support

        0111     1.0000    1.0000    1.0000         3
        0112     1.0000    0.5000    0.6667         2
        0120     1.0000    1.0000    1.0000         9
        0211     1.0000    1.0000    1.0000         9
        0212     0.7143    1.0000    0.8333         5
        0213     0.5000    0.5000    0.5000         2
        0220     1.0000    1.0000    1.0000       145
        0311     1.0000    1.0000    1.0000  

In [12]:
# ============================================================================
# 💾 GUARDADO COMPLETO DEL MODELO Y ARTEFACTOS
# ============================================================================

print("\n" + "="*80)
print("💾 GUARDANDO MODELO Y ARTEFACTOS")
print("="*80 + "\n")

try:
    config.logger.info("Guardando modelo final...")

    # 1. Guardar modelo y tokenizer
    trainer.save_model(config.MODEL_SAVE_DIR)
    tokenizer.save_pretrained(config.MODEL_SAVE_DIR)

    print(f"✅ Modelo y tokenizer guardados en: {config.MODEL_SAVE_DIR}")
    config.logger.info(f"Modelo guardado en: {config.MODEL_SAVE_DIR}")

    # 2. Guardar artefactos (mapeos, configuración, métricas)
    artifacts = {
        'label2id': label2id,
        'id2label': id2label,
        'num_labels': len(label2id),
        'target_column': config.TARGET_COLUMN,
        'text_column': config.TEXT_COLUMN,
        'max_length': config.MAX_LENGTH,
        'model_name': config.MODEL_NAME,
        'model_type': config.model_type,
        'experiment_name': config.experiment_name,
        'training_date': datetime.now().isoformat(),
        'test_metrics': test_metrics,
        'val_metrics': val_metrics if 'val_metrics' in locals() else None,
        'training_time': str(training_time) if 'training_time' in locals() else None,
        'device': config.DEVICE,
    }

    artifacts_file = os.path.join(config.OUTPUT_DIR, 'artifacts.pkl')
    with open(artifacts_file, 'wb') as f:
        pickle.dump(artifacts, f)

    print(f"✅ Artefactos guardados en: {artifacts_file}")
    config.logger.info(f"Artefactos guardados en: {artifacts_file}")

    # 3. Guardar resumen en JSON (legible)
    summary = {
        'experiment_name': config.experiment_name,
        'model_name': config.MODEL_NAME,
        'model_type': config.model_type,
        'num_classes': len(label2id),
        'training_date': datetime.now().isoformat(),
        'data_path': config.DATA_PATH,
        'max_length': config.MAX_LENGTH,
        'batch_size': config.BATCH_SIZE,
        'learning_rate': config.LEARNING_RATE,
        'num_epochs': config.NUM_EPOCHS,
        'test_accuracy': test_metrics.get('test_accuracy', test_metrics.get('eval_accuracy', 0)),
        'test_f1_macro': test_metrics.get('test_f1_macro', test_metrics.get('eval_f1_macro', 0)),
        'test_f1_weighted': test_metrics.get('test_f1_weighted', test_metrics.get('eval_f1_weighted', 0)),
        'device_used': config.DEVICE,
    }

    summary_file = os.path.join(config.OUTPUT_DIR, 'experiment_summary.json')
    with open(summary_file, 'w', encoding='utf-8') as f:
        json.dump(summary, f, indent=2, ensure_ascii=False)

    print(f"✅ Resumen guardado en: {summary_file}")

    # 4. Crear archivo README
    readme_content = f"""# Experimento: {config.experiment_name}

## Información del Modelo
- **Modelo Base**: {config.MODEL_NAME}
- **Tipo**: {config.model_type}
- **Número de Clases**: {len(label2id)}
- **Fecha de Entrenamiento**: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

## Resultados (Test Set)
- **Accuracy**: {test_metrics.get('test_accuracy', test_metrics.get('eval_accuracy', 0)):.4f}
- **F1 Macro**: {test_metrics.get('test_f1_macro', test_metrics.get('eval_f1_macro', 0)):.4f}
- **F1 Weighted**: {test_metrics.get('test_f1_weighted', test_metrics.get('eval_f1_weighted', 0)):.4f}
- **Precision Weighted**: {test_metrics.get('test_precision_weighted', test_metrics.get('eval_precision_weighted', 0)):.4f}
- **Recall Weighted**: {test_metrics.get('test_recall_weighted', test_metrics.get('eval_recall_weighted', 0)):.4f}

## Archivos Generados
- `final_model/`: Modelo entrenado y tokenizer
- `artifacts.pkl`: Mapeos y metadata
- `config.json`: Configuración del entrenamiento
- `test_metrics.json`: Métricas completas de test
- `classification_report.txt`: Reporte detallado por clase
- `error_analysis.csv`: Análisis de errores
- `experiment_summary.json`: Resumen del experimento

## Cómo Usar el Modelo

```python
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pickle

# Cargar modelo y tokenizer
model_path = "{config.MODEL_SAVE_DIR}"
model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Cargar artefactos
with open('{artifacts_file}', 'rb') as f:
    artifacts = pickle.load(f)

id2label = artifacts['id2label']

# Hacer predicción
text = "Tu texto aquí"
inputs = tokenizer(text, return_tensors='pt', max_length={config.MAX_LENGTH},
                   padding='max_length', truncation=True)
outputs = model(**inputs)
predicted_class = outputs.logits.argmax().item()
predicted_label = id2label[predicted_class]
print(f"Predicción: {{predicted_label}}")
```
"""

    readme_file = os.path.join(config.OUTPUT_DIR, 'README.md')
    with open(readme_file, 'w', encoding='utf-8') as f:
        f.write(readme_content)

    print(f"✅ README creado en: {readme_file}")

    # 5. Listar todos los archivos generados
    print("\n" + "="*80)
    print("📁 ARCHIVOS GENERADOS")
    print("="*80 + "\n")

    all_files = [
        ('Modelo final', config.MODEL_SAVE_DIR),
        ('Artefactos', artifacts_file),
        ('Configuración', os.path.join(config.OUTPUT_DIR, 'config.json')),
        ('Métricas de test', os.path.join(config.OUTPUT_DIR, 'test_metrics.json')),
        ('Reporte de clasificación', report_file),
        ('Análisis de errores', errors_file),
        ('Resumen del experimento', summary_file),
        ('README', readme_file),
        ('Info de clases', os.path.join(config.OUTPUT_DIR, 'class_info.json')),
    ]

    for name, path in all_files:
        if os.path.exists(path):
            if os.path.isdir(path):
                print(f"✅ {name:.<40} {path}")
            else:
                size = os.path.getsize(path) / 1024
                print(f"✅ {name:.<40} {path} ({size:.1f} KB)")

    print("\n" + "="*80 + "\n")

    config.logger.info("Todos los archivos guardados exitosamente")

except Exception as e:
    print("\n" + "="*80)
    print("❌ ERROR AL GUARDAR ARCHIVOS")
    print("="*80)
    print(f"\n{str(e)}\n")
    config.logger.error(f"Error al guardar: {str(e)}", exc_info=True)
    raise



💾 GUARDANDO MODELO Y ARTEFACTOS

✅ Modelo y tokenizer guardados en: /content/drive/MyDrive/Colab Notebooks/PI_PEU/Robertabne/bertin-roberta-base-spanish_20251112_225626/final_model
✅ Artefactos guardados en: /content/drive/MyDrive/Colab Notebooks/PI_PEU/Robertabne/bertin-roberta-base-spanish_20251112_225626/artifacts.pkl
✅ Resumen guardado en: /content/drive/MyDrive/Colab Notebooks/PI_PEU/Robertabne/bertin-roberta-base-spanish_20251112_225626/experiment_summary.json
✅ README creado en: /content/drive/MyDrive/Colab Notebooks/PI_PEU/Robertabne/bertin-roberta-base-spanish_20251112_225626/README.md

📁 ARCHIVOS GENERADOS

✅ Modelo final............................ /content/drive/MyDrive/Colab Notebooks/PI_PEU/Robertabne/bertin-roberta-base-spanish_20251112_225626/final_model
✅ Artefactos.............................. /content/drive/MyDrive/Colab Notebooks/PI_PEU/Robertabne/bertin-roberta-base-spanish_20251112_225626/artifacts.pkl (6.2 KB)
✅ Configuración........................... /content

In [13]:
# ============================================================================
# 🔮 FUNCIONES DE PREDICCIÓN E INFERENCIA
# ============================================================================

def load_trained_model_for_inference(model_dir, artifacts_path, device=None):
    """
    Carga el modelo entrenado para hacer predicciones

    Args:
        model_dir: Directorio del modelo guardado
        artifacts_path: Ruta al archivo de artefactos
        device: Dispositivo ('cuda' o 'cpu'), None para auto-detectar

    Returns:
        tuple: (model, tokenizer, artifacts)
    """
    import torch
    from transformers import AutoTokenizer, AutoModelForSequenceClassification
    import pickle

    if device is None:
        device = 'cuda' if torch.cuda.is_available() else 'cpu'

    print("\n" + "="*80)
    print("📂 CARGANDO MODELO PARA INFERENCIA")
    print("="*80 + "\n")

    # Cargar tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_dir)
    print(f"✅ Tokenizer cargado")

    # Cargar modelo
    model = AutoModelForSequenceClassification.from_pretrained(model_dir)
    model.to(device)
    model.eval()  # Modo evaluación
    print(f"✅ Modelo cargado en: {device}")

    # Cargar artefactos
    with open(artifacts_path, 'rb') as f:
        artifacts = pickle.load(f)
    print(f"✅ Artefactos cargados")
    print(f"   Número de clases: {artifacts['num_labels']}")

    print("\n" + "="*80 + "\n")

    return model, tokenizer, artifacts


def predict_single(text, model, tokenizer, artifacts, device=None, top_k=5):
    """
    Predice la clase de un texto individual

    Args:
        text: Texto a clasificar
        model: Modelo entrenado
        tokenizer: Tokenizer
        artifacts: Diccionario de artefactos
        device: Dispositivo
        top_k: Número de predicciones principales a retornar

    Returns:
        dict: Resultados de la predicción
    """
    import torch
    import torch.nn.functional as F

    if device is None:
        device = next(model.parameters()).device

    # Tokenizar
    inputs = tokenizer(
        text,
        max_length=artifacts['max_length'],
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Predicción
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = F.softmax(logits, dim=-1)

    # Predicción principal
    predicted_idx = torch.argmax(probs, dim=-1).item()
    predicted_prob = probs[0][predicted_idx].item()
    predicted_label = artifacts['id2label'][str(predicted_idx)]

    # Top-K predicciones
    top_probs, top_indices = torch.topk(probs[0], k=min(top_k, len(artifacts['id2label'])))
    top_predictions = [
        {
            'label': artifacts['id2label'][str(idx.item())],
            'probability': prob.item()
        }
        for prob, idx in zip(top_probs, top_indices)
    ]

    return {
        'text': text,
        'predicted_label': predicted_label,
        'predicted_probability': predicted_prob,
        'top_predictions': top_predictions
    }


def predict_batch(texts, model, tokenizer, artifacts, device=None, batch_size=32):
    """
    Predice las clases de múltiples textos

    Args:
        texts: Lista de textos
        model: Modelo entrenado
        tokenizer: Tokenizer
        artifacts: Diccionario de artefactos
        device: Dispositivo
        batch_size: Tamaño del lote

    Returns:
        list: Lista de predicciones
    """
    import torch
    import torch.nn.functional as F
    from tqdm.auto import tqdm

    if device is None:
        device = next(model.parameters()).device

    model.eval()
    predictions = []

    # Procesar en lotes
    for i in tqdm(range(0, len(texts), batch_size), desc="Prediciendo"):
        batch_texts = texts[i:i+batch_size]

        # Tokenizar
        inputs = tokenizer(
            batch_texts,
            max_length=artifacts['max_length'],
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        inputs = {k: v.to(device) for k, v in inputs.items()}

        # Predicción
        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits
            probs = F.softmax(logits, dim=-1)

        # Extraer predicciones
        predicted_indices = torch.argmax(probs, dim=-1).cpu().numpy()
        predicted_probs = torch.max(probs, dim=-1)[0].cpu().numpy()

        for idx, prob in zip(predicted_indices, predicted_probs):
            predictions.append({
                'predicted_label': artifacts['id2label'][str(idx)],
                'probability': float(prob)
            })

    return predictions


def interactive_prediction_mode(model, tokenizer, artifacts, device=None):
    """
    Modo interactivo para probar el modelo

    Args:
        model: Modelo entrenado
        tokenizer: Tokenizer
        artifacts: Diccionario de artefactos
        device: Dispositivo
    """
    print("\n" + "="*80)
    print("🧪 MODO DE PRUEBA INTERACTIVO")
    print("="*80)
    print("\nEscribe un texto para clasificar (o 'salir' para terminar)")
    print("Comandos especiales: 'salir', 'exit', 'quit', 'q'\n")

    while True:
        print("-" * 80)
        text = input("\n📝 Texto: ").strip()

        if text.lower() in ['salir', 'exit', 'quit', 'q']:
            print("\n👋 ¡Hasta luego!\n")
            break

        if not text:
            print("⚠️  Por favor, ingresa un texto válido")
            continue

        # Realizar predicción
        result = predict_single(text, model, tokenizer, artifacts, device)

        # Mostrar resultados
        print("\n📊 RESULTADO:")
        print(f"   🎯 Clase predicha: {result['predicted_label']}")
        print(f"   📈 Confianza: {result['predicted_probability']:.2%}")
        print(f"\n   🔝 Top 5 predicciones:")
        for i, pred in enumerate(result['top_predictions'], 1):
            bar = "█" * int(pred['probability'] * 20)
            print(f"      {i}. {pred['label']:<15} {pred['probability']:>6.2%} {bar}")
        print()


print("✅ Funciones de predicción cargadas")


✅ Funciones de predicción cargadas


In [14]:
# ============================================================================
# 💡 EJEMPLO DE USO - PREDICCIÓN CON EL MODELO ENTRENADO
# ============================================================================

# Este código muestra cómo usar el modelo después del entrenamiento
# Puedes ejecutarlo en este notebook o en uno nuevo

print("\n" + "="*80)
print("💡 EJEMPLOS DE USO")
print("="*80 + "\n")

# ============================================================================
# OPCIÓN 1: Usar el modelo recién entrenado (en este notebook)
# ============================================================================

print("📝 OPCIÓN 1: Predicción individual con modelo actual\n")

# Ejemplo de textos para probar
ejemplos_texto = [
    "vendedor de abarrotes en bodega",
    "profesor de matemáticas en colegio secundario",
    "conductor de taxi"
]

print("Ejemplos de predicción:\n")
for texto in ejemplos_texto:
    result = predict_single(texto, model, tokenizer, artifacts, config.DEVICE)
    print(f"Texto: {texto}")
    print(f"Predicción: {result['predicted_label']} (confianza: {result['predicted_probability']:.2%})\n")

print("-" * 80 + "\n")

# ============================================================================
# OPCIÓN 2: Cargar modelo guardado (en una nueva sesión)
# ============================================================================

print("📝 OPCIÓN 2: Código para cargar el modelo en una nueva sesión\n")

codigo_ejemplo = f'''
# Código para usar en un notebook/script nuevo:

# 1. Cargar el modelo
model_inference, tokenizer_inference, artifacts_inference = load_trained_model_for_inference(
    model_dir="{config.MODEL_SAVE_DIR}",
    artifacts_path="{os.path.join(config.OUTPUT_DIR, 'artifacts.pkl')}"
)

# 2. Hacer predicción individual
texto = "albañil de construcción"
result = predict_single(
    text=texto,
    model=model_inference,
    tokenizer=tokenizer_inference,
    artifacts=artifacts_inference
)
print(f"Predicción: {{result['predicted_label']}}")
print(f"Confianza: {{result['predicted_probability']:.2%}}")

# 3. Hacer predicción por lotes
textos = ["texto 1", "texto 2", "texto 3"]
predictions = predict_batch(
    texts=textos,
    model=model_inference,
    tokenizer=tokenizer_inference,
    artifacts=artifacts_inference,
    batch_size=32
)

# 4. Modo interactivo
interactive_prediction_mode(
    model=model_inference,
    tokenizer=tokenizer_inference,
    artifacts=artifacts_inference
)
'''

print(codigo_ejemplo)

print("=" * 80 + "\n")



💡 EJEMPLOS DE USO

📝 OPCIÓN 1: Predicción individual con modelo actual

Ejemplos de predicción:



KeyError: '193'

In [ ]:
# ============================================================================
# 🎮 MODO INTERACTIVO - PROBAR EL MODELO
# ============================================================================

# Descomenta y ejecuta esta celda para probar el modelo interactivamente

# interactive_prediction_mode(
#     model=model,
#     tokenizer=tokenizer,
#     artifacts=artifacts,
#     device=config.DEVICE
# )

print("\n💡 Descomenta el código arriba para activar el modo interactivo\n")


---

# 🎉 ¡ENTRENAMIENTO COMPLETADO!

## 📊 Resumen de Resultados

Tu modelo ha sido entrenado exitosamente. Revisa:

1. **Métricas de Test**: Accuracy, F1, Precision, Recall (macro, micro, weighted)
2. **Archivos Generados**: Todos los archivos están en el directorio de salida
3. **README**: Instrucciones detalladas de uso

## 🚀 Próximos Pasos

### Para entrenar otro modelo:
1. Cambia `MODEL_NAME` en `ModelConfig`
2. Ejecuta todas las celdas nuevamente

### Para usar el modelo:
1. Usa las funciones de predicción en este notebook
2. O carga el modelo en un script nuevo (ver ejemplos arriba)

### Para mejorar los resultados:
- Ajusta los hiperparámetros en `ModelConfig`
- Aumenta `NUM_EPOCHS`
- Experimenta con diferentes `LEARNING_RATE`
- Aumenta `MAX_LENGTH` si tus textos son largos

---

## 📚 Modelos Soportados

Este script funciona con:
- ✅ `FacebookAI/xlm-roberta-base` (Multilingüe)
- ✅ `dccuchile/bert-base-spanish-wwm-cased` (Español)
- ✅ Cualquier modelo de Hugging Face compatible con `AutoModelForSequenceClassification`

---

**¿Preguntas o errores?** Revisa:
- El archivo de log en el directorio de salida
- Los mensajes de error detallados en cada celda
- La documentación de transformers: https://huggingface.co/docs/transformers
